# Benchmarking the code for analysis with artificial data

created 2025-03-26, by Kananovich
updated 2025-04-17 by Kananovich

In [1]:
import numpy as np
import scipy.stats as sts
from scipy import constants as cnst
import pandas as pd

general parameters.
camera resolution updated on 2025-04-17 using the data provided by Parth

In [2]:
nP = 1000 #number of particles
rho = 1510.0
dust_diam = 7.14E-6
fps = 1.0E10 # camera framerate in frames per second
#fps = 295.0 # camera framerate in frames per second
time_step = 1.0 / fps
res_meters_per_px = 30.0E-20
#res_meters_per_px = 30E-6
resol_SI = 1.0 / res_meters_per_px # camera resolution in px/meters
dust_mass = 4.0 / 3.0 * np.pi * (dust_diam / 2.0)**3 * rho #mass of the dust particles
kin_Tx = 1000.0 #kinetic temperature (in Kelvins) along the x axis
drift_x = 0  # asuming the average x-component of the particles is zero (no drift)
left_x_bord = 0
right_x_bord = 1751.0 # right border of the field of view in pixels
left_x_bord_SI = left_x_bord / resol_SI
right_x_bord_SI = right_x_bord / resol_SI #coordinated of the right border
    #of the filed of view in meters


In [3]:
resol_SI

3.3333333333333335e+18

## Step 1. Creating an array of artificial velocities

In [4]:
arr_ind = np.arange(0,nP,1,dtype = 'int') # array of particles ID numbers

In [5]:
sigma_x = np.sqrt(cnst.k * kin_Tx / dust_mass)

In [6]:
sigma_x

0.00021903148058823087

In [7]:
art_vx_rv = sts.norm(drift_x,sigma_x)
arr_sample_vx = art_vx_rv.rvs(nP)

## Step 2. Creating an array of artificial coordinates

In [8]:
art_x_prev_rv = sts.uniform(left_x_bord_SI, right_x_bord_SI - left_x_bord_SI)
arr_sample_prev_x = art_x_prev_rv.rvs(nP)


In [9]:
arr_prev_x_inResolChunks = arr_sample_prev_x * resol_SI

In [10]:
arr_prev_x_inResolChunks_int = arr_prev_x_inResolChunks.astype(int)

In [11]:
arr_rough_prev_x = arr_prev_x_inResolChunks_int.astype('float64') / resol_SI


## Step 3. Creating an array of artificial coordinates for the "next frame"

In [12]:
arr_next_x = arr_rough_prev_x + arr_sample_vx * time_step

In [13]:
arr_next_x_inResolChunks = arr_next_x * resol_SI
arr_next_x_inResolChunks_int = arr_next_x_inResolChunks.astype('int64')
arr_rough_next_x = arr_next_x_inResolChunks_int.astype('float64') / resol_SI

## Step 4. Calculating the restored velocities

In [14]:
arr_vx_restored = (arr_rough_next_x - arr_rough_prev_x) / time_step

## Step 5. Calculating the array of discrepancies

In [15]:
arr_discrep_x = np.abs(arr_vx_restored - arr_sample_vx)

In [16]:
arr_frac_discrep_x = np.abs(arr_discrep_x / arr_sample_vx) * 100

In [17]:
arr_frac_discrep_x.min()

2.5881810864227027e-06

In [18]:
arr_frac_discrep_x.max()

0.13749412538010675

In [19]:
arr_frac_discrep_x.mean()

0.0031373405157319265

In [5]:
def create_art_vels(tTx, tTy, N_particles, metersPerPx, frps, ro, diam, drift_x, drift_y, left_x, right_x, lefty, right_y):
    import numpy as np
    import scipy.stats as sts
    from scipy import constants as cnst
    resol_SI = 1.0 / metersPerPx # camera resolution in px/meters
    arr_ind = np.arange(0,N_particles,1,dtype = 'int') # array of particles ID numbers
    dust_mass = 4.0 / 3.0 * np.pi * (diam / 2.0)**3 * ro #mass of the dust particles
    sigma_x = np.sqrt(cnst.k * tTx / dust_mass)
    sigma_y = np.sqrt(cnst.k * tTy / dust_mass)
    left_x_bord_SI = left_x / resol_SI
    right_x_bord_SI = right_x / resol_SI #coordinated of the right border
    time_step = 1.0 / frps

    

    art_vx_rv = sts.norm(drift_x,sigma_x)
    arr_sample_vx = art_vx_rv.rvs(N_particles)

    #Array of artificial coordinates for the "previous" frame:
    art_x_prev_rv = sts.uniform(left_x_bord_SI, right_x_bord_SI - left_x_bord_SI)
    arr_sample_prev_x = art_x_prev_rv.rvs(N_particles)

    arr_prev_x_inResolChunks = arr_sample_prev_x * resol_SI
    arr_prev_x_inResolChunks_int = arr_prev_x_inResolChunks.astype(int)
    arr_rough_prev_x = arr_prev_x_inResolChunks_int.astype('float64') / resol_SI
    
    ## Step 3. Creating an array of artificial coordinates for the "next frame"
    arr_next_x = arr_rough_prev_x + arr_sample_vx * time_step
    arr_next_x_inResolChunks = arr_next_x * resol_SI
    arr_next_x_inResolChunks_int = arr_next_x_inResolChunks.astype('int64')
    arr_rough_next_x = arr_next_x_inResolChunks_int.astype('float64') / resol_SI

    arr_ind_bothFrames = np.concatenate((arr_ind,arr_ind))
    arr_x_both_frames = np.concatenate((arr_sample_prev_x,arr_rough_next_x))

    #saving all the data in the output dataframe:
    #data = {'frame':arr_frame, 'particle':arr_particle, 'x': arr_x, 'y': arr_y, 'vx': arr_vx, 'vy':arr_vy}
    #ret_df = pd.DataFrame(arr_ind_bothFrames, columns = ["particle"])
    data = {'particle':arr_ind_bothFrames, 'x': arr_x_both_frames}
    ret_df = pd.DataFrame(data)
 
    
    
    return ret_df
test = create_art_vels(1200, 1200, 1000, 1.0E-6, 100, 1510, 7.14E-6, 0, 0, 0, 1751, 0, 400)
test

,particle,x
0,0,0.000549
1,1,0.001013
2,2,0.000975
3,3,0.001694
4,4,0.001706
...,...,...
1995,995,0.000094
1996,996,0.000694
1997,997,0.000595
1998,998,0.000842


In [15]:
array1 = np.array([1.1, 2.1, 3.1])
array2 = np.array([4, 5, 6])
result = np.concatenate((array1, array2))
print(result)

[1.1 2.1 3.1 4.  5.  6. ]
